In [1]:
import os
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import LineString

# ─────────────────────────────────────────────
# Paths
# ─────────────────────────────────────────────
OUTPUT_DIR = "../output/"
INPUT_DIR = "../input/"

# ─────────────────────────────────────────────
# Load route GeoJSONs
# ─────────────────────────────────────────────
route_files = [f for f in os.listdir(OUTPUT_DIR) if f.endswith("_tracks.geojson")]
routes_list = []

for file in route_files:
    gdf = gpd.read_file(os.path.join(OUTPUT_DIR, file))
    city_code = file.replace("_tracks.geojson", "")
    gdf["city_code"] = city_code
    routes_list.append(gdf)

routes_gdf = pd.concat(routes_list, ignore_index=True)
print(f"✅ Loaded {len(routes_gdf)} segments from {len(route_files)} files")

# ─────────────────────────────────────────────
# Try to load stations
# ─────────────────────────────────────────────
try:
    stations_df = pd.read_csv(os.path.join(OUTPUT_DIR, "stations.csv"))
    print(f"✅ Loaded {len(stations_df)} stations")
except FileNotFoundError:
    stations_df = None
    print("⚠️  No stations.csv found. Skipping station markers.")

# ─────────────────────────────────────────────
# Create base map centered on Lebanon
# ─────────────────────────────────────────────
center_lat = 33.9
center_lon = 35.5

m = folium.Map(location=[center_lat, center_lon], zoom_start=8, tiles="OpenStreetMap")

# Optional basemaps
folium.TileLayer("CartoDB positron", name="Light").add_to(m)
folium.TileLayer("Stamen Terrain", name="Terrain").add_to(m)
folium.LayerControl().add_to(m)

# ─────────────────────────────────────────────
# Add routes to map
# ─────────────────────────────────────────────
for _, row in routes_gdf.iterrows():
    geom = row.geometry
    if isinstance(geom, LineString):
        folium.PolyLine(
            locations=[(lat, lon) for lon, lat in geom.coords],
            color="blue",
            weight=4,
            tooltip=f"Route: {row.get('city_code', '')}"
            
        ).add_to(m)

# ─────────────────────────────────────────────
# Add stations (if any)
# ─────────────────────────────────────────────
if stations_df is not None:
    for _, row in stations_df.iterrows():
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=5,
            color="red",
            fill=True,
            tooltip=row.get("city_name", "station")
        ).add_to(m)

# ─────────────────────────────────────────────
# Display map (or save as HTML)
# ─────────────────────────────────────────────
m.save("../output/network_map.html")
m


✅ Loaded 7 segments from 7 files
⚠️  No stations.csv found. Skipping station markers.
